# **Integrating Databases**
This is our final regular assignment for the course. This notebook will demonstrate how you can connect to a SQL Server database using a programming language, and integrate that data into your program. For this demo we will be using Python since it is compatible with Jupyter Notebooks and is somewhat easy to read. You will not be expected to write ANY python for this assignment, you may have to mimic a process, but at most you will be copy/pasting and updating query scripts.

To connect to a database using Python we must import the pyodbc library.

### **Installing pyodbc**

1. In VS Code use ctrl + ` (the key to the left of 1) to open the terminal
2. Enter this command and hit enter: *pip install pyodbc* | If that does not work, try one of these
  - py -m pip install pyodbc
  - python -m pip install pyodbc
  - python3 -m pip install pyodbc
3. Once the terminal executes the command you are good to go. If you have trouble please reach out to your instructor.

In [1]:
# Import Needed Libraries
import pyodbc

When connecting to a database using a programming language, we usually need to use a *connection string*. A connection string is made up of the information needed to connect to a database, like the server, database name, and login information. The below cell stores information in a variable called connection_string that we can use to access the database you have been using all semester (besides recent modules.)

Update the below cell to use the connection information outline in the SQL Server connection Info page in Canvas.

* UID is same as login
* PWD is password

In [13]:
# Define connection string
connection_string = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=cisfa24-2.ckwia8qkgyyj.us-east-1.rds.amazonaws.com;'  # Replace with your server name
    'DATABASE=<ms0885011>;'  # Replace with your database name
    'UID=<Ms0885011>;'  # Replace with your username
    'PWD=<0885011>'  # Replace with your password
)

In the next cell, we are going to use the pyodbc module to try to establish a connection to the database. If everything in the previous cell was done correctly, then this cell should return "Connection Successful". If you get an error message, review the information you entered above.

In [12]:
# Establish Connection
try:
    connection = pyodbc.connect(connection_string)
    print('Connection Successful')
except pyodbc.Error as ex:
    sqlstate = ex.args[1]
    print(f'Connection error: {sqlstate}')

Connection error: [IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)


In Python, we use what's called a *cursor* object to interact with the database. The cursor contains functions for executing scripts, committing changes, and fetching records. We will be using all of these. The below cell will create a cursor object for us to use.

In [11]:
# Create a cursor object using the connection
cursor = connection.cursor()

NameError: name 'connection' is not defined

Now that all of the setup is done, we can finally start running some queries. To start, we are going to write out our query and save it to a variable. We are using what is called an r-string, it stands for *raw-string literal*. When we put an r in front of our string, it disables escape and special characters so that the string is interpreted exactly as it is typed.

The query that is in the below cell is returning all records from the products table in the guitar store system where the product_id is one. We use the execute() function of our cursor object to run the query. Running execute stores the returned data in the memory of the cursor.

The fetchall() function is then used to pull all of the rows from the memory of the cursor and assign them to the rows variable that we set up. We know from experience that this query will only return one record, however, the fetchall() function always returns a list of rows, so to display the data we must loop through the returned list and output the data using print().

The goal is not for you to be able to write Python to do something like this yourself, the goal is for you to examine the way that a different programming language interacts with SQL so that you can diagnose an issue or a need in a program, and then write the needed SQL script to resolve the need. In short, you only need to be able to read the Python and follow along, don't worry about writing anything on your own.

In [ ]:
# Example query
query = r'SELECT * FROM gs_products WHERE product_id = 1'

# Execute the query
cursor.execute(query)

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Print row
for row in rows:
    print(row)

By this time you should have been exposed to an object-oriented programming language (C#, Java, or Python), and know what a class is. If you need a refresher, classes are a programming tool that defines a collection of properties and functions to reduce redundancy in code by creating objects. In the below example I have defined a product class using Python. After running the below cell we will be able to create product objects in our code based off of the data in our database. This will make it much easier to interact with the data in our program.

Note that I am creating a property in my class for each attribute in the related table.

In [ ]:
# Create a class to contain the contents of the row in a Python object
class product:
    def __init__(self, product_id,category_id,product_code,product_name,product_description,list_price,discount_percent,date_added):
        self.product_id = product_id
        self.category = category_id
        self.product_code = product_code
        self.product_name = product_name
        self.product_description = product_description
        self.list_price = list_price
        self.discount_percent = discount_percent
        self.date_added = date_added

Now that our class is setup, we are going to use the same loop that we used earlier to save the data from cursor to a python object named product_info. We do this by passing each row property into the product constructor. We defined the constructor in the above cell using def __ init __, if you would like to review it.

Run the below cell and read through it carefully so that you can get a feel for what is happening:
* Looping through each row in the rows object from earlier
* Creating an object named product_info
* Setting the product_info object equal to a new instance of the product class
* Product class constructor is using the data from the row that is currently selected by the for loop (There is only one row in the list, so the loop is only running once.)

In [ ]:
# Save the contents of the row to a Python object
for row in rows:
    product_info = product(row.product_id,row.category_id,row.product_code,row.product_name,row.product_description,row.list_price,row.discount_percent,row.date_added)

Now we can display a piece of information from our database using regular dot-notation to access the product_name property of our product_info object. All of this seems like a long process, but can be automated with functions and done very quickly. In a real program we would hide our SQL operations in functions that we could then call to make accessing the database extremely simple.

In [ ]:
# Display the product name by accessing the class property
print(product_info.product_name)

Speaking of simplicity, instead of writing entire queries to pull data into a program, we can write stored procedures that take parameters than will make our code much cleaner. In the below cell we are executing a query to create a stored procedure that we will call instead of hard-coding in a query. Review the SQL script that we are executing and note what it is doing.

In [ ]:
# Create a stored procedure to query for a product when given the product id
query = r'CREATE PROCEDURE get_product_by_id @input_product_id INT AS BEGIN SELECT * FROM gs_products WHERE product_id = @input_product_id END'

cursor.execute(query)

cursor.commit()

Since the above procedure accepts a parameter for product_id, this means that we can use a variable in our program to specify the product_id that we want to pull a product for. In the below cell we declare and assign a value to the product_id variable (4). In the following line we call our stored procedure using the EXEC keyword. Note that we are using an f-string instead of an r-string. This is because we are going to be incorporating the value of a variable into our query string. Instead of saying:

EXEC get_product_by_id @input_product_id=4

We are using a variable so that our code is more flexible and can be changed on the fly. This adds another layer of abstraction to our process. In a full program we would do the following:
1. Write a stored procedure to pull data
2. Write a function in our app to call the stored procedure, accepting product_id as an argument
3. Get the product_id from the user or another processes
4. Call the function that accesses the stored procedure and provide the selected product_id

We are doing some of these steps, but spelling them out very deliberately so that we can examine the process.

After the procedure is ran, we fetch all of the records (There is just one again). Then we save the data to an object and access the properties of that object to display the data.

In [ ]:
# Use the stored procedure to pull data into an object
product_id = 4 # Specify product_id

query = f'EXEC get_product_by_id @input_product_id={product_id}'

cursor.execute(query)

rows = cursor.fetchall()

for row in rows:
    product_info = product(row.product_id,row.category_id,row.product_code,row.product_name,row.product_description,row.list_price,row.discount_percent,row.date_added)

print(f'The product related to id number {product_id} is: {product_info.product_name}.')

Run the above cell again, but change the value of the product_id variable in the first line of code to be 5 or another number. This demonstrates how seamless it can be to pull data once we have everything set up correctly.

The pyodbc library is very powerful. We can execute pretty much any SQL command using the execute() method. In the below cell we add a new record to the product table by writing an INSERT query and executing it.

In [ ]:
# Write a query to add new data to the product table
query = r"INSERT INTO gs_products VALUES (1,'yama','Yamaha F335 Acoustic','The F335 acoustic guitar from Yamaha is a stellar blend of quality, playability and affordability.',189.99,10,'2024-07-15')"

cursor.execute(query)

cursor.commit()

By integrating our database deeply into our programming we can even store each attribute value that needs inserted in it's own variable. Once stored we can use those variables in an f-string to construct our query like below. It looks like a lot to write, but you only have to set it up once, and then you can call this same script each time that you need to insert a record. All you would need to do it change the variable values to insert different data.

In a full program this data would usually come from something a user types into a form, so once the program is developed, your involvement can be scaled back.

Note that in the below cell we include ' (single quotes) within the " (double quotes) when assigning values to our variables. This is because the SQL script we are saving in our query variable would expect for their to be single quotes around the values. We can include the single quotes in the variable values, or in the SQL statement. It does not matter which as long as it gets done.

In [ ]:
# Inserting new data using variables instead of a hard-coded string
category_id = 3
product_code = r"'ludw'"
product_name = r"'Ludwig BackBeat Elite'"
product_description = r"'Ludwig’s BackBeat drum set was a hit for giving drummers a rock-solid foundation for their rhythmic aspirations'"
list_price = 799
discount_percent = 15
date_added = r"'2024-07-16'"

# Use the variables to construct the query
query = f'INSERT INTO gs_products VALUES({category_id},{product_code},{product_name},{product_description},{list_price},{discount_percent},{date_added})'

# Execute and commit the query
cursor.execute(query)

cursor.commit()

Now, let's go ahead and display all of the rows in the product table so that we can verify the insertion of our two new records.

In [ ]:
# Display all records in the gs_products table

# Construct query
query = r'SELECT * FROM gs_products'

# Execute the query
cursor.execute(query)

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Print row
for row in rows:
    print(row)

### **Exercises**

For the below exercises you will be writing SQL scripts and procedures to accompany the provided Python script. If it is easier, feel free to type up your SQL in Azure Data Studio and then paste it into the exercise.
* Script all necessary procedures before running the provided Python script.
* Create a new code cell for each procedure you are adding if it helps you stay organized.
* If you successfully run an insert/delete statement, but need to run the python code an additional time to see if everything is working correctly, comment out the execution of the query that performs the addition/deletion.

1. Review the below python script to assess the database needs. Then, in the following cell add two stored procedures to your database that will enable the python script to run correctly. You may **not** change anything in the python script.

In [ ]:
# Script to add a new category to the categories table and display all records. Do Not Change.
category_name = 'Woodwinds'

query_1 = f'EXEC add_new_category @category_name = {category_name}'
query_2 = f'EXEC show_categories'

# Add new record
cursor.execute(query_1)

# Display all records
rows = cursor.execute(query_2)

for row in rows:
    print(row)

In [ ]:
# Once again cannot get the server to connect nor run properly. Typing these out and hoping for the best. 
# 
# procedure 1
# ereate procedure
query = r'''
CREATE PROCEDURE add_new_category @category_name NVARCHAR(255) 
AS 
BEGIN 
    IF NOT EXISTS ( 
        SELECT 1 
        FROM categories 
        WHERE category_name = @category_name
    ) 
    BEGIN 
        INSERT INTO categories (category_name) 
        VALUES (@category_name); 
    END 
    ELSE 
    BEGIN 
        PRINT 'Category already exists.'; 
    END 
END
'''

# execute procedure
cursor.execute(query)
cursor.commit()


# procedure 2
query2 = r'''
CREATE PROCEDURE show_categories
AS 
BEGIN 
    SELECT category_id, category_name 
    FROM categories 
    ORDER BY category_name;
END
'''

# execute procedure
cursor.execute(query2)
cursor.commit()





NameError: name 'cursor' is not defined

2. Review the below python script to assess the database needs. Then, in the following cell add two stored procedures to your database that will enable the python script to run correctly. You may **not** change anything in the python script.

In [ ]:
# Script to delete an item from an order when provided the item_id and display all records to verify deletion. Do Not Change
item_id = 45

query_1 = f'EXEC del_item_order @item_id = {item_id}'
query_2 = f'EXEC show_order_items'

# Delete a record
cursor.execute(query_1)

# Display all records
rows = cursor.execute(query_2)

for row in rows:
    print(row)

In [ ]:

# procedure 1
# create procedure 
query = '''
CREATE PROCEDURE del_item_order @item_id INT
AS
BEGIN
    DELETE FROM order_items WHERE item_id = @item_id;
END
'''

# execute the procedure
cursor.execute(query)
cursor.commit()

# procedure 2
# create procedure
query2 = '''
CREATE PROCEDURE show_order_items
AS
BEGIN
    SELECT item_id, item_name, quantity, price FROM order_items ORDER BY item_id;
END
'''

# execute procedure
cursor.execute(query2)
cursor.commit()



3. Review the below python script to assess the database needs. Then, in the following cell add a stored procedure to your database that will enable the python script to run correctly. You may **not** change anything in the python script. Read through the script carefully to determine what fields your procedure should return.

In [ ]:
# Script to show the information about a customer that placed an order. Do Not Change.
order_id = 10

query_1 = f'EXEC get_order_info @order_id = {order_id}'

# Retrieve order data
cursor.execute(query_1)

rows = cursor.fetchall()

# Display Order data in nice format
for row in rows:
    print(f'Order#: {row.order_id}\t{row.order_date}\nCustomer: {row.first_name} {row.last_name}\nEmail Address: {row.email_address}')

In [ ]:
# procedure 1
# create procedure
query = '''
CREATE PROCEDURE get_order_info @order_id INT
AS
BEGIN
    SELECT o.order_id, o.order_date, c.first_name, c.last_name, c.email_address
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    WHERE o.order_id = @order_id;
END
'''

# execute procedure
cursor.execute(query)
cursor.commit()


NameError: name 'cursor' is not defined

### **Scenario**

Review the below python script to assess the database needs. Then, in the following cell add the required stored procedure(s) to your database that will enable the python script to run correctly. You may **not** change anything in the python script. Don't forget that you can comment out query executions if they have ran successfully and you need to run queries further down in the cell.

To assist with this exercise, below are the statements used to create the customer and address tables. Double click this cell for better readability:

CREATE TABLE gs_customers (<br>
  customer_id           INT            PRIMARY KEY   IDENTITY,<br>
  email_address         VARCHAR(255)   NOT NULL      UNIQUE,<br>
  customer_password     VARCHAR(60)    NOT NULL,<br>
  first_name            VARCHAR(60)    NOT NULL,<br>
  last_name             VARCHAR(60)    NOT NULL,<br>
  shipping_address_id   INT                          DEFAULT NULL,<br>
  billing_address_id    INT                          DEFAULT NULL<br>
);<br>

CREATE TABLE gs_addresses (<br>
  address_id          INT            PRIMARY KEY   IDENTITY,<br>
  customer_id         INT            REFERENCES gs_customers (customer_id),<br>
  line1               VARCHAR(60)    NOT NULL,<br>
  line2               VARCHAR(60)                  DEFAULT NULL,<br>
  city                VARCHAR(40)    NOT NULL,<br>
  state_code          VARCHAR(2)     NOT NULL,<br>
  zip_code            VARCHAR(10)    NOT NULL,<br>
  phone               VARCHAR(12)    NOT NULL,<br>
  disabled_address     INT            NOT NULL      DEFAULT 0<br>
);



In [ ]:
# Script to add a new customer and address to the database
email_address = 'test@email.com'
customer_password = 'SecurePassword'
first_name = 'Test'
last_name = 'Testerman'
line1 = '123 Test Street'
line2 = 'Apt 123'
city = 'Testville'
state_code = 'MO'
zip_code = '99999'
phone = '1234567890'

# Query to add a customer to the database
query_1 = f"EXEC add_customer @email_address = '{email_address}',@customer_password = '{customer_password}',@first_name = '{first_name}',@last_name = '{last_name}'"

# Run and commit query 1
cursor.execute(query_1)

cursor.commit()


# Query to get the id of a customer using their email
query_2 = f"EXEC get_customer_id @email_address = '{email_address}'"


# Store the value from query 2 in a variable
cursor.execute(query_2)

rows = cursor.fetchall()

for row in rows:
    customer_id = row.customer_id


# Query to add an address to the database
query_3 = f"EXEC add_address @customer_id = {customer_id},@line1 = '{line1}',@line2 = '{line2}',@city = '{city}',@state_code = '{state_code}',@zip_code = '{zip_code}',@phone = '{phone}'"

# Run and commit query 3
cursor.execute(query_3)

cursor.commit()

# Verify insertion of address record by displaying all addresses
query_4 = f"EXEC get_addresses"

cursor.execute(query_4)

rows = cursor.fetchall()

for row in rows:
    print(row)


In [ ]:

# add customer procedure 
query = '''
CREATE PROCEDURE add_customer 
    @email_address VARCHAR(255),
    @customer_password VARCHAR(60),
    @first_name VARCHAR(60),
    @last_name VARCHAR(60)
AS
BEGIN
    INSERT INTO gs_customers (email_address, customer_password, first_name, last_name)
    VALUES (@email_address, @customer_password, @first_name, @last_name);
END
'''

# execute procedure
cursor.execute(query)
cursor.commit()

# get customer id procedure
query2 = '''
CREATE PROCEDURE get_customer_id 
    @email_address VARCHAR(255)
AS
BEGIN
    SELECT customer_id FROM gs_customers WHERE email_address = @email_address;
END
'''

# execute
cursor.execute(query2)
cursor.commit()



# add address procedure
query3 = '''
CREATE PROCEDURE add_address 
    @customer_id INT,
    @line1 VARCHAR(60),
    @line2 VARCHAR(60),
    @city VARCHAR(40),
    @state_code VARCHAR(2),
    @zip_code VARCHAR(10),
    @phone VARCHAR(12)
AS
BEGIN
    INSERT INTO gs_addresses (customer_id, line1, line2, city, state_code, zip_code, phone)
    VALUES (@customer_id, @line1, @line2, @city, @state_code, @zip_code, @phone);
END
'''

# execute
cursor.execute(query3)
cursor.commit()



# add get addresses procedure
query4 = '''
CREATE PROCEDURE get_addresses
AS
BEGIN
    SELECT address_id, customer_id, line1, line2, city, state_code, zip_code, phone 
    FROM gs_addresses
    WHERE disabled_address = 0;
END
'''

# execute
cursor.execute(query4)
cursor.commit()



In [ ]:
# Use this cell to delete procedures if you make a mistake
query_list = 'DROP PROCEDURE add_customer','DROP PROCEDURE get_customer_id','DROP PROCEDURE add_address','DROP PROCEDURE get_addresses'

for query in query_list:
    cursor.execute(query)
    cursor.commit()